# Programmation d'un modèle de langue n-gram

# Import des packages

In [1]:
import numpy as np

## Traitement des données d'apprentissage

On utilise ici l'extrait assez faible (15 Mo) disponible sur le drive de Benjamin.

In [2]:
fichier = open("fr_wikipedia_sample.txt","r",encoding="utf8")
corpus=fichier.read()
fichier.close()

In [3]:
print(corpus[:500])

Paul Jules Antoine Meillet , né le 11 novembre 1866 à Moulins ( Allier ) et mort le 21 septembre 1936 à Châteaumeillant ( Cher ) , est le principal linguiste français des premières décennies du XXe siècle .
Il est aussi philologue .
Biographie D' origine bourbonnaise , fils d' un notaire de Châteaumeillant ( Cher ) , il fait ses études secondaires au lycée de Moulins .
Étudiant à la faculté des lettres de Paris à partir de 1885 où il suit notamment les cours de Louis Havet , il assiste également


In [7]:
#corpus = corpus.replace(",","").replace("-","").replace("'","' ")

In [4]:
#Virer la casse
corpus = corpus.lower()

#Virer la ponctuation et la remplacer par les balises <s> et </s>
corpus = corpus.replace(".","</s> <s>").replace("!","</s> <s>").replace("?","</s> <s>").replace("\n"," ")
corpus = "<s> "+corpus
#Enlever l'espace à la fin
corpus = corpus[:len(corpus)-1]

Interrogation : Faut-il traiter les virgules, parenthèses et autres symboles comme des mots ?

In [5]:
seqcorpus = corpus.split(' ')

In [6]:
seqcorpus[:9]

['<s>', 'paul', 'jules', 'antoine', 'meillet', ',', 'né', 'le', '11']

In [7]:
vocabulaire = list(set(seqcorpus))

In [8]:
print(len(vocabulaire))

113895


## Apprentissage des fréquences des n-grams

In [81]:
class NGramModel:
    
    def __init__(self, n):
        self.n = n
        self.vocabulary = []
        self.freq_n_minus_one_grams = {}
        self.freq_n_grams = {}
    
    @staticmethod
    def counting_kgrams(text_tokens, k):
        freq_kgrams = {}
        for i in range(0,len(text_tokens)-k+1):
            kgram = ' '.join(text_tokens[i:i+k])
            freq_kgrams[kgram] = freq_kgrams.get(kgram,0)+1
        return freq_kgrams
        
    def fit(self, text_tokens):
        self.vocabulary = list(set(text_tokens))
        self.freq_n_minus_one_grams = self.counting_kgrams(text_tokens, self.n-1)
        self.freq_n_grams = self.counting_kgrams(text_tokens, self.n)
    
    def predict_probas(self, word, words_pred):
        if self.freq_n_minus_one_grams.get(words_pred, 0) == 0:
            return 0
        return (self.freq_n_grams.get(words_pred + " " + word,0) 
                / self.freq_n_minus_one_grams.get(words_pred,0))
    
    def generate_greedy(self, nb_words_togen, words_pred):
        for i in range(nb_words_togen):
            probas_cond=[]
            for word in self.vocabulary:
                probas_cond.append(self.predict_probas(word," ".join(words_pred.split(' ')[-(self.n-1):])))
            #Selectionne le mot qui maximise la probabilité conditionnelle (greedy search)
            words_pred += " " + self.vocabulary[np.argmax(probas_cond)]
        return words_pred
    
    

# Tests Greedy

In [79]:
for i in range(2,6):
    print("Résultats pour",i,"gram")
    test = NGramModel(i)
    test.fit(seqcorpus)
    print(test.generate_greedy(50,"<s> il est le"))

Résultats pour 2 gram
<s> il est un des années 1980 , le plus de la première fois , le plus de la première fois , le plus de la première fois , le plus de la première fois , le plus de la première fois , le plus de la première fois , le plus de
Résultats pour 3 gram
<s> il est le plus souvent , les deux pays </s> <s> le premier ministre , qui est le plus souvent , les deux pays </s> <s> le premier ministre , qui est le plus souvent , les deux pays </s> <s> le premier ministre , qui est le plus souvent , les
Résultats pour 4 gram
<s> il est le premier à avoir utilisé le terme « aviation » , bien que le président de la république , il désigna celui qui commandait l’ armée </s> <s> le premier , le second degré 541 , soit un total de 7 nouveaux gisements , qui se déroule dans un asile


## Utilisation pour la génération de texte

Remarque : ça se met souvent à boucler ! Pour éviter ça, on peut introduire une part d'aléatoire dans le choix du mot suivant, par exemple en tirant selon la distribution de probabilité plutôt qu'en choisissant systématiquement le plus probable.

Beam Search Method

In [123]:
#Génère une séquence de nbmots mots conditionnellement à la séquence précédente seqprec
#Avec la méthode Beam Search pour k meilleurs séquences conservées à chaque étape
def genererbeam(nbmots,seqprec,k):
    sequences=[[seqprec,1]]
    #A chaque etape
    for i in range(nbmots):
        #calcule tous les candidats possibles de l'étape (sequence totale et proba associée)
        candidates=[]
        for j in range(len(sequences)):
            seq, prob = sequences[j]
            for mot in vocabulaire:
                candidates.append([seq+" "+mot,prob*probacond(mot," ".join(seq.split(' ')[-(n-1):]))])
        # ordonne les candidats selon le score
        ordered = sorted(candidates, key=lambda tup:tup[1])
        # sélectionne les k meilleurs
        sequences = ordered[-k:]
    return sequences

In [104]:
#Cas bigram
genererbeam(30,"<s>",3)

[['<s> le premier ministre de la fin de la ville de la ville de la ville de la ville de la ville de la ville de la ville de la ville',
  7.486873168181089e-34],
 ['<s> le premier ministre de la fin de la ville de la ville de la ville de la ville de la ville de la ville de la ville de la première',
  8.241045373767097e-34],
 ['<s> le premier ministre de la fin de la ville de la ville de la ville de la ville de la ville de la ville de la première fois , le',
  9.530640308839064e-34]]

In [116]:
#Cas trigram
genererbeam(30,"<s> il",3)

[["<s> il s' agit d' un point de vue de la population </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> la première",
  2.592672590660988e-27],
 ["<s> il s' agit d' un point de vue de la population </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> le pays",
  2.841209628240636e-27],
 ["<s> il s' agit d' un point de vue de la population </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> le premier",
  3.1430881512412038e-27]]

In [124]:
#Cas quatrigram
genererbeam(50,"<s> il est",3)

[["<s> il est également possible de porter la balle à deux mains </s> <s> lance : terme générique désignant une arme offensive dotée d' un fer emmanché sur une hampe </s> <s> jetés à terre , leurs corps sont foulés par les vainqueurs représentés sous des formes mixtes , en partie grâce aux",
  2.0868169263891186e-11],
 ["<s> il est également possible de porter la balle à deux mains </s> <s> lance : terme générique désignant une arme offensive dotée d' un fer emmanché sur une hampe </s> <s> jetés à terre , leurs corps sont foulés par les vainqueurs représentés sous des formes mixtes , en partie , par",
  2.608521157986398e-11],
 ["<s> il est également possible de porter la balle à deux mains </s> <s> lance : terme générique désignant une arme offensive dotée d' un fer emmanché sur une hampe </s> <s> jetés à terre , leurs corps sont foulés par les vainqueurs représentés sous des formes mixtes , en partie grâce à",
  8.347267705556474e-11]]

In [ ]:
#Cas 5-gram
genererbeam(30,"<s> il est le",3)

Améliorations :
-> lissage (smoothing)
-> Apprendre les fréquences de tous les k-grams pour k<n pour pouvoir switcher à un k plus petit si le k-gram recherché est absent lors de la prédiction
-> Travailler avec les log-probas pour être sûr de ne pas perdre en précision -> Le temps pour générer le texte me semble très long, il y a sans doute moyen d'optimiser le code